# 1. packages

In [ ]:
using Flux
import Pkg; Pkg.add("CUDA")
using CUDA

In [ ]:
using CSV, DataFrames, JudiLing, RCall

# 2. dataset

In [ ]:
mp = JudiLing.load_dataset("../dat/mp.txt", delim=" ");
mp[:,"features"] = string.(mp.Hub, "_", mp.Neighbor);
first(mp, 10)

X and Y are the coordinates on an integer grid. The property to be predicted is in the column *Outcome* (binary version *R*). A datapoint is a *Hub* when it is surrounded completely by 8 points with outcome "yes". A datapoint is a *Neighbor* when it shares a "yes" datapoint with a *Hub*.

In [ ]:
RCall.rcall_p(:options, rcalljl_options=Dict(:width => 600, :height => 600))
@rput mp;
R"""
plot(mp$X, mp$Y, col=ifelse(mp$R==1, "indianred", "steelblue2"), xlab="x", ylab="y")
""";

# 3. Linear mappings

## 3.1 A linear mapping using X and Y doesn't work.

A GLM given coordinates transformed into factors gets closest.

In [ ]:
R"""
mp$Xf=factor(mp$X)
mp$Yf=factor(mp$Y)
mp$Outcome = factor(mp$Outcome)
mp.glm = glm(Outcome~ Xf+Yf, data=mp, family="binomial")
mp$glm_predict = predict(mp.glm, type="response")

mp$correct_pred = (mp$R==1 & mp$glm_predict > 0.5) | 
                  (mp$R==0 & mp$glm_predict < 0.5)
mpx = mp[!mp$correct_pred,]

plot(mp$X, mp$Y, col=ifelse(mp$R==1, "indianred", "steelblue2"), xlab="x", ylab="y")
points(mpx$X, mpx$Y, pch=19)
""";

## 3.2 A linear mapping using Hub and Neighbor features is successful.

In [ ]:
a = JudiLing.make_pS_matrix(mp, features_col=:features);
C = a.pS;
a.i2f

In [ ]:
size(C)

In [ ]:
S = JudiLing.make_pS_matrix(mp, features_col="Outcome").pS;
first(Array(S),6)

In [ ]:
F = JudiLing.make_transform_matrix(C, S);
Shat = C * F;
v = Shat .> 0.5
sum(v[:,1] .== S)/size(S, 1)

# 4. Nonlinear mappings using deep learning

We ask deep networks to predict the four feature values of Hub and Neighbor. If these are learned correctly, then prediction of the Outcome will be perfect.  In other words, can a deep network learn the mathematically optimal representation of the problem?

In [ ]:
Input = Matrix(mp[:, 1:2]);
# the coordinates
Output = Array(C);
# the four feature values

## 4.1 Hyperparameters settings that do not work

### 4.1.1 a model with a single hidden layer

In [ ]:
model_test = Chain(
Dense(size(Input, 2) => 1000, relu),
Dense(1000 => size(Output, 2)),
sigmoid) |> gpu

In [ ]:
using Random
Random.seed!(314);

Training this model takes about 30 seconds.

In [ ]:
m1 = JudiLing.get_and_train_model(
            Input,
            Output,
            "../res/m1.bcomp",
            verbose=true,
            loss_func=Flux.binarycrossentropy,
            n_epochs=1000,
            model = model_test)

In [ ]:
losses = m1.losses_train
@rput losses
R"""
plot(1:length(losses), losses, type="l", xlab="epoch", ylab="loss", col="steelblue2")
""";

The loss remains highly variable.  Performance at the end of training:

In [ ]:
m1_pred = JudiLing.predict_from_deep_model(m1.model, Input)
@rput m1_pred;

In [ ]:
R"""
library(MASS)
m1.lda = lda(m1_pred, mp$Outcome)
tab = table(mp$Outcome, predict(m1.lda)$class)
tab
"""

In [ ]:
R"""
mp$m1_correct = predict(m1.lda)$class==mp$Outcome
plot(mp$X, mp$Y, col=ifelse(mp$Outcome=="yes", "indianred", "steelblue2"), xlab="x", ylab="y")
mpx = mp[!mp$m1_correct,]
points(mpx$X, mpx$Y, col="black", pch=19)
""";

This model fails at the boundary, and at the center.

### 4.1.2 a model with two large hidden layers

In [ ]:
model_test = Chain(
Dense(size(Input, 2) => 1000, relu),
Dense(1000 => 1000, relu),
Dense(1000 => size(Output, 2)), sigmoid) |> gpu

In [ ]:
Random.seed!(314);

This model takes about 90 minutes to train.

In [ ]:
m2 = JudiLing.get_and_train_model(
            Input,
            Output,
            "../res/m2.bcomp",
            verbose=true,
            loss_func=Flux.binarycrossentropy,
            optimizer=Flux.Adam(0.0001),
            n_epochs=1000,
            model = model_test)

In [ ]:
losses = m2.losses_train
@rput losses
R"""
plot(1:length(losses), losses, type="l", xlab="epoch", ylab="loss", col="steelblue2", ylim=c(0,1))
""";

In [ ]:
R"""
pdf("../fig/losses_large_model_doughnut.pdf", he=5, wi=5)
plot(1:length(losses), losses, type="l", xlab="epoch", ylab="loss")
dev.off()
""";

The loss remains highly variable.  Performance at the end of training:

In [ ]:
m2_pred = JudiLing.predict_from_deep_model(m2.model, Input)
@rput m2_pred;

In [ ]:
R"""
library(MASS)
m2.lda = lda(m2_pred, mp$Outcome)
tab = table(mp$Outcome, predict(m2.lda)$class)
tab
"""

In [ ]:
R"""
mp$m2_correct = predict(m2.lda)$class==mp$Outcome
plot(mp$X, mp$Y, col=ifelse(mp$Outcome=="yes", "indianred", "steelblue2"), xlab="x", ylab="y")
mpx = mp[!mp$m2_correct,]
points(mpx$X, mpx$Y, col="black", pch=19)
""";

After 1.5 hours of training, with more than 1 million parameters, the model still makes errors at the boundary of the red area.

## 4.1 Hyperparameters that do work more efficiently

In [ ]:
Random.seed!(314);

model_test = Chain(
Dense(size(Input, 2) => 20, relu),
Dense(20 => 10, relu),
Dense(10 => 4, relu),
Dense(4 => 10, relu),
Dense(10 => 20, relu),
Dense(20 => 10, relu),
Dense(10 => size(Output, 2)), sigmoid) |> gpu

Training this model takes about 5.5 minutes.

In [ ]:
m3 = JudiLing.get_and_train_model(
        Input,
        Output, 
        "../res/m3.bcomp",
        verbose=true,
        loss_func=Flux.binarycrossentropy,
        n_epochs=6000,
        optimizer=Flux.Adam(0.0001),
        model = model_test);

m3_pred = JudiLing.predict_from_deep_model(m3.model, Input);
@rput m3_pred;

Classification accuracy is almost perfect:

In [ ]:
R"""
m3.lda = lda(m3_pred, mp$Outcome)
tab = table(mp$Outcome, predict(m3.lda)$class)
tab
"""

In [ ]:
R"""
mp$m3_correct = predict(m3.lda)$class==mp$Outcome
plot(mp$X, mp$Y, col=ifelse(mp$Outcome=="yes", "indianred", "steelblue2"), xlab="x", ylab="y")
mpx = mp[!mp$m3_correct,]
points(mpx$X, mpx$Y, col="black", pch=19)
""";

The loss function is very noisy, though:

In [ ]:
losses = m3.losses_train
@rput losses
R"""
plot(1:length(losses), losses, type="l", xlab="epoch", ylab="loss", col="steelblue2")
""";

In [ ]:
R"""
pdf("../fig/losses_small_deepnetwork_doughnut.pdf", he=5, wi=5)
plot(1:length(losses), losses, type="l", xlab="epoch", ylab="loss")
dev.off()
""";

With a different seed, we might be lucky and achieve 100% accuracy.

In [ ]:
Random.seed!(2718);

model_test = Chain(
Dense(size(Input, 2) => 20, relu),
Dense(20 => 10, relu),
Dense(10 => 4, relu),
Dense(4 => 10, relu),
Dense(10 => 20, relu),
Dense(20 => 10, relu),
Dense(10 => size(Output, 2)), sigmoid) |> gpu

Training this model takes 5.5 minutes.

In [ ]:
m4 = JudiLing.get_and_train_model(
        Input,
        Output, 
        "../res/m3.bcomp",
        verbose=true,
        loss_func=Flux.binarycrossentropy,
        n_epochs=6000,
    optimizer=Flux.Adam(0.0001),
        model = model_test);

m4_pred = JudiLing.predict_from_deep_model(m4.model, Input);
@rput m4_pred;

And indeed:

In [ ]:
R"""
m4.lda = lda(m4_pred, mp$Outcome)
tab = table(mp$Outcome, predict(m4.lda)$class)
tab
"""

In [ ]:
losses = m4.losses_train
@rput losses
R"""
plot(1:length(losses), losses, type="l", xlab="epoch", ylab="loss", col="steelblue2")
""";